In [32]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q  http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-07 00:32:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.43MB/s    in 0.6s    

2020-08-07 00:33:00 (1.43 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level2").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [34]:
# Read in data from S3 files
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
lawn_gard_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True)
lawn_gard_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [35]:
# Count rows - before cleanup
print("Total product review count before cleanup: ",lawn_gard_df.count() )

Total product review count before cleanup:  2557288


In [36]:
# Drop null values
lawn_gard_df = lawn_gard_df.dropna()
lawn_gard_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [37]:
# Count rows - after cleanup
print("Number of product reviews after cleanup: ",lawn_gard_df.count())

Number of product reviews after cleanup:  2557005


In [38]:
# Get Product vote info 
product_voter_df = lawn_gard_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
product_voter_df.show(10)
product_voter_df.count()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|            2|          8|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            4|          5|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            5|          6|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            2|          2|   N|                Y|
|          3|            0|          0|   N|                Y|
|          2|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



2557005

In [39]:
# Get total verified and Unverified purchases
total_verified_df = lawn_gard_df.filter(lawn_gard_df['verified_purchase'] == 'Y')
total_unverified_df = lawn_gard_df.filter(lawn_gard_df['verified_purchase'] == 'N')

print("Total verified purchases: ",total_varified_df.count())
print("Total Unverified purchases: ",total_unvarified_df.count())
print("% Ratio of Unverified to verified purchase reviews: ",float(total_unvarified_df.count()/total_varified_df.count()))

Total verified purchases:  2251611
Total Unverified purchases:  305394
% Ratio of Unverified to verified purchase reviews:  0.1356335530426881


In [40]:
# Get the products with 10 plus total votes
total_vote_df = lawn_gard_df.filter(lawn_gard_df['total_votes'] >= 10)
total_vote_df.show(10)
print("Number of products with reviews with at least 10 votes: ",total_vote_df.count())

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33399595| R6J125A9S5H1G|B00OMSJ9WG|     583972217|10x10 V-Series 2 ...| Lawn and Garden|          5|            9|         10|   N|                Y|Straight leg cano...|We bought 2 of th...| 2015-08-31|
|         US|   12020967|R3SJT43TE6IY0O|B00S96Q2UO|     997068254|Heat Resistant Si...| Lawn and Garden|          3|    

In [41]:
# Percentage of products with 10 plus total votes for the review
print("Percentage of products with 10 plus reviews: ",float(total_vote_df.count()/lawn_gard_df.count()),"%")

Percentage of products with 10 plus reviews:  0.049227123138202704 %


In [42]:
# Describe stats for paid and unpaid products 
from pyspark.sql.functions import col, avg
paid_df = lawn_gard_df.filter(lawn_gard_df['vine']== 'Y')
unpaid_df = lawn_gard_df.filter(lawn_gard_df['vine']== 'N')

paid_df.describe().show()
unpaid_df.describe().show()

+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|product_id|     product_parent|       product_title|product_category|      star_rating|     helpful_votes|       total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|  count|      13454|               13454|         13454|     13454|              13454|               13454|           13454|            13454|             13454|             13454|13454|            13454|           

In [43]:
#  Determine paid and five star paid reviews 
paid_number = paid_df.count()
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()

print("Paid Review count: ",paid_number)
print("Paid Five Star Review count: ",paid_five_star_number)

Paid Review count:  13454
Paid Five Star Review count:  6006


In [44]:
#  Determine the percentage of five-star reviews among Vine reviews
percentage_five_star_vine = paid_five_star_number/paid_number

print("% of five-star reviews among Vine reviews",float(percentage_five_star_vine),"%")

% of five-star reviews among Vine reviews 0.4464099895941727 %


In [45]:
#  Determine the percentage of five-star reviews among non-Vine reviews.
unpaid_number = unpaid_df.count()
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()

print("Unpaid Review count: ",paid_number)
print("Unpaid Five Star Review count: ",paid_five_star_number)

Unpaid Review count:  13454
Unpaid Five Star Review count:  6006


In [46]:
# Determine the percentage of five-star reviews among non-Vine reviews.
percentage_five_star_non_vine = unpaid_five_star_number/unpaid_number
print("% of five-star unpaid reviews among non Vine reviews", float(percentage_five_star_non_vine),"%")

% of five-star unpaid reviews among non Vine reviews 0.605333645757447 %
